In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# 신뢰구간
from scipy.stats import t, sem

# 1. 모수 검정
# 1-1. 정규성
from scipy.stats import shapiro, kstest, normaltest, anderson

# 1-2. 등분산성
from scipy.stats import bartlett, levene, fligner

# 1-3. t-test
from scipy.stats import ttest_1samp, ttest_ind, ttest_rel

# 1-4. ANOVA(분산분석)
from scipy.stats import f_oneway

# 2. 비모수 검정
# 2-1. 카이제곱 적합도 검정
from scipy.stats import chisquare

# 2-2. 카이제곱 동질성/독립성 검정
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact

# 2-3. 중앙값을 사용한 비모수 검정
from scipy.stats import wilcoxon, ranksums, mannwhitneyu, kruskal

# 2-4. 피셔의 정확 검정
from scipy.stats import fisher_exact

# 1. 모수 검정

## 1) 정규성 검정

### 1-1) 정규성 검정의 종류 
- **Shapiro-Wilks Test:** 표본 수가 5,000개 미만인 데이터셋에 적합한 정규성 검정
- **Kolmogorov-Smirnov Test:** 표본 수가 2,000개 초과인 데이터셋에 적합한 정규성 검정
- **Normal Test:** 왜도와 첨도를 통해 정규성 검정, 20개 이상의 데이터 필요- Anderson Darling Test: 표본 수가 5,000개 초과인 데이터셋에 적합한 정규성 검정, 5개의 유의 수준에 대한 임곗값 반환

In [2]:
df = pd.read_csv('./data_02/sleepage.csv')
print(df.head())

   stime20s  stime40s  ID
0         4         5   1
1         4         5   2
2         5         6   3
3         5         7   4
4         6         6   5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   stime20s  20 non-null     int64
 1   stime40s  20 non-null     int64
 2   ID        20 non-null     int64
dtypes: int64(3)
memory usage: 608.0 bytes


In [4]:
# [1] 그룹 나누기
gA = df['stime20s']
gB = df['stime40s']

# [2] 정규성 검정 - shaprio
shapiro_sA, shapiro_pA = shapiro(gA)
shapiro_sB, shapiro_pB = shapiro(gB)

# [2] 정규성 검정 - kstest
ks_sA, ks_pA = kstest(gA, 'norm') 
ks_sB, ks_pB = kstest(gB, 'norm')

# [3] 정규성 검정 - normaltest
normal_sA, normal_pA = normaltest(gA)
normal_sB, normal_pB = normaltest(gB)

# [4] 정규성 검정 - anderson
rA = anderson(gA) # statistic, critical_values, significance_level
rB = anderson(gB) # statistic, critical_values, significance_level

print('shapiro 정규성 검정 결과')
print(f'그룹A - statistic: {shapiro_sA:.04f} p-value: {shapiro_pA:.04f}')
print(f'그룹B - statistic: {shapiro_sB:.04f} p-value: {shapiro_pB:.04f}\n')

print('kstest 정규성 검정 결과')
print(f'그룹A - statistic: {ks_sA:.04f} p-value: {ks_pA:.04f}')
print(f'그룹B - statistic: {ks_sB:.04f} p-value: {ks_pB:.04f}\n')

print('normaltest 정규성 검정 결과')
print(f'그룹A - statistic: {normal_sA:.04f} p-value: {normal_pA:.04f}')
print(f'그룹B - statistic: {normal_sB:.04f} p-value: {normal_pB:.04f}\n')

print('anderson 정규성 검정 결과')
print(f'그룹A -', *rA)
print(f'그룹B -', *rB)

shapiro 정규성 검정 결과
그룹A - statistic: 0.9239 p-value: 0.1180
그룹B - statistic: 0.8810 p-value: 0.0184

kstest 정규성 검정 결과
그룹A - statistic: 0.9987 p-value: 0.0000
그룹B - statistic: 1.0000 p-value: 0.0000

normaltest 정규성 검정 결과
그룹A - statistic: 1.1732 p-value: 0.5562
그룹B - statistic: 2.0450 p-value: 0.3597

anderson 정규성 검정 결과
그룹A - 0.5601097602856164 [0.506 0.577 0.692 0.807 0.96 ] [15.  10.   5.   2.5  1. ]
그룹B - 0.8918678522034931 [0.506 0.577 0.692 0.807 0.96 ] [15.  10.   5.   2.5  1. ]


## 2) 등분산성 검정

### 2-1) 등분산성 검정의 종류
- **Shapiro-Wilks Test:** 표본 수가 5,000개 미만인 데이터셋에 적합한 정규성 검정
- **Kolmogorov-Smirnov Test:** 표본 수가 2,000개 초과인 데이터셋에 적합한 정규성 검정
- **Normal Test:** 왜도와 첨도를 통해 정규성 검정, 20개 이상의 데이터 필요
- **Anderson Darling Test:** 표본 수가 5,000개 초과인 데이터셋에 적합한 정규성 검정, 5개의 유의 수준에 대한 임곗값 반환

In [5]:
df = sns.load_dataset('iris')
print(df.head())

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [7]:
# [1] 그룹 나누기1 
# target = 'sepal_length'  
# gA = df.loc[df['species']=='setosa', target].to_list()
# gB = df.loc[df['species']=='versicolor', target].to_list()
# gC = df.loc[df['species']=='virginica', target].to_list()

# [1] 그룹 나누기2
groups = [x.to_list() for name, x in df.groupby('species')['sepal_length']]
# groups = [x.to_list() for name, x in df.groupby('species')['sepal_length']]
gA, gB, gC = groups
# print(gA)

# [2] 정규성 검정
_, pA = shapiro(gA)
_, pB = shapiro(gB)
_, pC = shapiro(gC)
print(pA, pB, pC)

# [3] 등분산성 검정 - bartlett
bartlett_s, bartlett_p = bartlett(gA, gB, gC)
print(bartlett_s, bartlett_p)

# [4] 등분산성 검정 - levene
levene_s, levene_p = levene(gA, gB, gC, center='mean')
# levene_s, levene_p = levene(*groups, center='mean')
print(levene_s, levene_p)

# [4] 등분산성 검정 - fligner
fligner_s, fligner_p = fligner(*groups, center='trimmed', proportiontocut=0.05)
print(fligner_s, fligner_p)

0.4595010578632355 0.4647378921508789 0.25831347703933716
16.005701874401502 0.0003345076070163035
7.381091747801267 0.0008817887814641548
13.193438547424174 0.0013648383616752843


## 3) T-검정

- 검정 통계량이 귀무가설 하에서 t-분포를 따르는 통계적 가설 검정
- 표본을 사용한 모평균 검정 및 두 데이터 세트(집단)의 모평균이 서로 유의하게 다른지 여부를 판별할 수 있음
- 검정통계량이 정규분포를 따르며 모집단의 분산, 표준편차를 알지 못할 때 표본으로부터 추정된 분산/표준편차를 사용해 검정함
- t-test를 실시하기 위해서는 정규성 및 등분산성의 조건이 만족되어야 함 

### 3-1) One sample t-test
- 표본을 사용한 모평균 검정 방법
- 예) 귀무가설: S사의 USB의 평균 수명은 20,000시간이다.

In [8]:
df = pd.read_csv("./data_02/sleepage.csv")
print(df.head())

   stime20s  stime40s  ID
0         4         5   1
1         4         5   2
2         5         6   3
3         5         7   4
4         6         6   5


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   stime20s  20 non-null     int64
 1   stime40s  20 non-null     int64
 2   ID        20 non-null     int64
dtypes: int64(3)
memory usage: 608.0 bytes


In [10]:
# [1] 20대의 수면 시간에 대한 정규성 검정
# one sample이므로 등분산 검정이 필요 없음
statistic, pvalue = shapiro(df['stime20s'])
print(f'statistic: {statistic:.04f} p-value:{pvalue:.04f}')
print('기각' if pvalue < 0.05 else '채택\n')

# [2] 20대 수면 시간에 대한 평균 구하기
# print(df['stime20s'].mean())

# [3] 가설 검정 - ttest_1samp
# 귀무가설: 20대 수면 시간의 평균은 6시간이다.
# 대립가설: 20대 수면 시간의 평균은 6시간이 아니다.

at_s, at_p = ttest_1samp(df['stime20s'], 6, alternative='two-sided')
print(f'statistic: {at_s:.04f} p-value: {at_p:.04f}')
print('기각' if at_p < 0.05 else '채택\n')

al_s, al_p = ttest_1samp(df['stime20s'], 6, alternative='less')
print(f'statistic: {al_s:.04f} p-value: {al_p:.04f}')
print('기각' if al_p < 0.05 else '채택\n')

ag_s, ag_p = ttest_1samp(df['stime20s'], 6, alternative='greater')
print(f'statistic: {ag_s:.04f} p-value: {ag_p:.04f}')
print('기각' if ag_p < 0.05 else '채택\n')

# [4] 95% 신뢰구간 구하기
target = df['stime20s']
lower, upper = t(df=len(target)-1).interval(0.95, loc=target.mean(), scale=sem(target))
print(f'{lower:.04f} ~ {upper:.04f}')

statistic: 0.9239 p-value:0.1180
채택

statistic: 0.4592 p-value: 0.6513
채택

statistic: 0.4592 p-value: 0.6743
채택

statistic: 0.4592 p-value: 0.3257
채택

5.4663 ~ 6.8337


### 3-2) Two-sample t-test
- 서로 다른 두 그룹의 표본 평균을 비교하여 두 모집단의 평균 차이가 있는지 검정하는 방법
- `equal_var = False`인 경우 Welch’s t-test를 수행

In [11]:
df = pd.read_csv('./data_02/sleepage.csv')
print(df.head())

   stime20s  stime40s  ID
0         4         5   1
1         4         5   2
2         5         6   3
3         5         7   4
4         6         6   5


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   stime20s  20 non-null     int64
 1   stime40s  20 non-null     int64
 2   ID        20 non-null     int64
dtypes: int64(3)
memory usage: 608.0 bytes


In [13]:
# [1] 그룹 나누기
gA = df.loc[:, 'stime20s']
gB = df.loc[:, 'stime40s']

# [2] 정규성 검정
sA, pA = shapiro(gA)
sB, pB = shapiro(gB)
print(f'{pA:.04f} {pB:.04f}')

# [3] 등분산성 검정
levene_s, levene_p     = levene(gA, gB)
bartlett_s, bartlett_p = bartlett(gA, gB)
fligner_s, fligner_p   = fligner(gA, gB)
print(f'{levene_p:.04f} {bartlett_p:.04f} {fligner_p:.04f}')

# [4] 가설 검정 - ttest_ind
at_s, at_p = ttest_ind(gA, gB, alternative='two-sided')
print(f'statistic: {at_s:.04f} p-value: {at_p:.04f}')
print('기각' if at_p < 0.05 else '채택\n')

al_s, al_p = ttest_ind(gA, gB, alternative='less')
print(f'statistic: {al_s:.04f} p-value: {al_p:.04f}')
print('기각' if al_p < 0.05 else '채택\n')

ag_s, ag_p = ttest_ind(gA, gB, alternative='greater')
print(f'statistic: {ag_s:.04f} p-value: {ag_p:.04f}')
print('기각' if ag_p < 0.05 else '채택\n')

0.1180 0.0184
0.2312 0.1475 0.1808
statistic: -0.4988 p-value: 0.6208
채택

statistic: -0.4988 p-value: 0.3104
채택

statistic: -0.4988 p-value: 0.6896
채택



정규성을 만족하지 않으면 비모수 검정의 Wilocoxon rank sum test,Mann-Whitney U test 등을 사용하는 것이 좋다. 예제는 우선 t-test로 진행했다.

### 3-3) Paired t-test
- 동일 개체에 어떤 처리를 하기 전, 후의 자료를 얻을 때 차이 값에 대한 평균 검정을 위한 방법
- 가능한 동일한 특성을 갖는 두 개체에 서로 다른 처리를 하여 그 처리의 효과를 비교하는 방법
- 예) 매일 1시간씩 한 달 걸으면 2kg이 빠진다(걷기 수행 전/수행 후)
- 예) X질병 환자들을 두 집단으로 나누어 A, B 약을 투약해 약의 효과 비교

In [14]:
df = pd.read_csv('./data_02/sleep.csv')
print(df.head())

   extra  group  ID
0    0.7      1   1
1   -1.6      1   2
2   -0.2      1   3
3   -1.2      1   4
4   -0.1      1   5


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   extra   20 non-null     float64
 1   group   20 non-null     int64  
 2   ID      20 non-null     int64  
dtypes: float64(1), int64(2)
memory usage: 608.0 bytes


In [16]:
# [1] 그룹 나누기
gA = df.loc[df.group==1, 'extra']
gB = df.loc[df.group==2, 'extra']

# [2] 정규성 검정
sA, pA = shapiro(gA)
sB, pB = shapiro(gB)
print(f'그룹A - statistic: {shapiro_sA:.04f} p-value: {shapiro_pA:.04f}')
print(f'그룹B - statistic: {shapiro_sB:.04f} p-value: {shapiro_pB:.04f}\n')

# [3] 등분산성 검정 n>=20
print(normaltest(gA))
print(normaltest(gB), '\n')
      
# [4] 가설 검정 - ttest_rel
at_s, at_p = ttest_rel(gA, gB, alternative='two-sided')
print(f'statistic: {at_s:.04f} p-value: {at_p:.04f}')
print('기각' if at_p < 0.05 else '채택\n')

al_s, al_p = ttest_rel(gA, gB, alternative='less')
print(f'statistic: {al_s:.04f} p-value: {al_p:.04f}')
print('기각' if al_p < 0.05 else '채택\n')

ag_s, ag_p = ttest_rel(gA, gB, alternative='greater')
print(f'statistic: {ag_s:.04f} p-value: {ag_p:.04f}')
print('기각' if ag_p < 0.05 else '채택\n')

그룹A - statistic: 0.9239 p-value: 0.1180
그룹B - statistic: 0.8810 p-value: 0.0184

NormaltestResult(statistic=0.8834543265697302, pvalue=0.6429250250799996)
NormaltestResult(statistic=1.969150292024901, pvalue=0.373597917789166) 

statistic: -4.0621 p-value: 0.0028
기각
statistic: -4.0621 p-value: 0.0014
기각
statistic: -4.0621 p-value: 0.9986
채택



C:\Users\futur\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


### 3-4) 예제

다음은 22명의 학생들이 국어시험에서 받은 점수이다. 학생들의 평균이 75보다 크다고 할 수 있는가?
- 모집단은 정규분포를 따른다.
- 표본의 크기가 충분히 크다.

In [3]:
# 귀무가설: 모평균은 mu와 같다. (μ = mu), 학생들의 평균은 75이다
# 대립가설: 모평균은 mu보다 크다. (μ > mu), 학생들의 평균은 75보다 크다

scores = [75, 80, 68, 72, 77, 82, 81, 79, 70, 74, 76, 78, 81, 73, 81, 78, 75, 72, 74, 79, 78, 79]

# 모평균 가설검정
mu = 75       # 검정할 모평균
alpha = 0.05  # 유의수준

statistic, pvalue = ttest_1samp(scores, mu, alternative='greater')

print(round(statistic, 4))
print(round(pvalue, 4))
print('기각' if pvalue < 0.05 else '채택')

1.7659
0.046
기각


A 회사에서 판매하는 모니터는 평균 5개 보다 적은 불량화소를 포함한다고 주장한다. 이 주장을 판단하기 위해서 데이터를 수집했으며, 주어진 데이터(data_02/defective.csv)에는 모니터 25개에서 조사한 불량화소 개수가 저장되어 있다. 불량화소의 개수는 정규분포를 따른다고 할 때, 이 주장의 타당성 여부를 유의수준 5%에서 검정하여라.

- 𝑯_𝟎: 𝝁≥𝟓, 𝑯_𝟏: 𝝁<𝟓, defective_pixel: 불량화소 개수
- (a) 불량화소의 표본 평균을 구하시오 (반올림하여 소수 둘째자리까지 계산)
- (b) 위의 가설을 검정하기 위한 검정통계량을 입력하시오.(반올림하여 소수 넷째자리까지 계산)
- (c) 위의 통계량에 대한 p-값을 구하여 입력하시오. (반올림하여 소수 넷째자리까지 계산)
- (d) 유의수준 0.05 하에서 가설검정의 결과를 (채택/기각) 중 하나를 선택하여 입력하시오.

In [17]:
# 귀무가설: A 회사에서 판매하는 모니터의 불량화소 평균이 5개보다 크거나 같다.
# 대립가설: A 회사에서 판매하는 모니터의 불량화소 평균이 5개보다 적다.

df = pd.read_csv('data_02/defective.csv')
# print(df.head())

print(round(df['defective_pixel'].mean(), 2))

statistic, pvalue = ttest_1samp(df['defective_pixel'], 5, alternative='less')

print(round(statistic, 4))
print(round(pvalue, 4))

print('기각' if pvalue < 0.05 else '채택')

4.84
-1.4446
0.0807
채택


주어진 데이터(data_02/blood_pressure.csv)에는 고혈압 환자 120명의 치료 전후의 혈압이 저장되어 있다. 해당 치료가 효과가 있는지 (즉, 치료 후의 혈압이 감소했는지) 쌍체표본 t-검정(paired t-test)를 통해 답하고자 한다. 가설은 아래와 같다.
- 𝑯_𝟎: 𝝁_𝒅≥𝟎, 𝑯_𝟏: 𝝁_𝒅<𝟎  (𝝁_𝒅: 치료 후 혈압 – 치료 전 혈압)의 평균
- bp_before : 치료 전 혈압, bp_after : 치료 후 혈압
- (a) ud의 표본 평균을 구하시오 (반올림하여 소수 둘째자리까지 계산)
- (b) 위의 가설을 검정하기 위한 검정통계량을 입력하시오.(반올림하여 소수 넷째자리까지 계산)
- (c) 위의 통계량에 대한 p-값을 구하여 입력하시오. (반올림하여 소수 넷째자리까지 계산)
- (d) 유의수준 0.05 하에서 가설검정의 결과를 (채택/기각) 중 하나를 선택하여 입력하시오.


In [18]:
df = pd.read_csv('data_02/blood_pressure.csv')
# print(df.head())

df['ud'] = df['bp_after'] - df['bp_before']

print(round(df['ud'].mean(), 2))

statistic, pvalue = ttest_rel(df['bp_after'], df['bp_before'], alternative='less')

print(round(statistic, 4))
print(round(pvalue, 4))
print('기각' if pvalue < 0.05 else '채택')

-5.09
-3.3372
0.0006
기각


어떤 특정 약물을 복용한 사람들의 평균 체온이 복용하지 않은 사람들의 평균 체온과 유의미하게 다른지 검정해보려고 한다.
- 약물을 복용한 그룹과 복용하지 않은 그룹의 체온 데이터가 각각 주어져 있다고 가정한다.
- 각 그룹의 체온은 정규분포를 따른다고 가정한다.

In [6]:
group1 = [36.8, 36.7, 37.1, 36.9, 37.2, 36.8, 36.9, 37.1, 36.7, 37.1]
group2 = [36.5, 36.6, 36.3, 36.6, 36.9, 36.7, 36.7, 36.8, 36.5, 36.7]

# 가설검정
statistic, pvalue = ttest_ind(group1, group2)

print(round(statistic, 4))
print(round(pvalue, 4))
print('기각' if pvalue < 0.05 else '채택')

3.7964
0.0013
기각


## 4) ANOVA(분산 분석)

- 독립변수 – 범주형, 종속변수 – 연속형으로 하는 3개 이상의 집단 평균을 비교하는 모수 추론 방법
- 집단 간 분산/집단 내 분산 기반의 F 분포를 이용해 가설을 검정하며 정규성, 등분산성, 독립성을 가정함
- 귀무가설 기각 시 구체적인 차이를 파악하기 위해, 사후 검정(Post Hoc Test)이 필요함

### 4-1) 일원분산분석(One-way ANOVA)
- 독립변수의 변화가 종속변수에 미치는 영향이 있는지를 알기 위해 사용
- 범주형 독립변수가 한 개인 경우 사용
- 예) 계절별 아이스크림 판매량 평균이 동일하다

In [19]:
df = pd.read_csv('bigdata/iris_data.csv')
df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'target']
print(df.head())

   sepal_length  sepal_width  petal_length  petal_width  target
0           5.1          3.5           1.4          0.2       0
1           4.9          3.0           1.4          0.2       0
2           4.7          3.2           1.3          0.2       0
3           4.6          3.1           1.5          0.2       0
4           5.0          3.6           1.4          0.2       0


In [20]:
# 정규성 확인
feature = 'sepal_width'

data = [x[1].values for x in df.groupby('target')[feature]]

_, pvalue0 = shapiro(data[0])
_, pvalue1 = shapiro(data[1])
_, pvalue2 = shapiro(data[2])
print(f'{pvalue0:.4f} {pvalue1:.4f} {pvalue2:.4f}')

# 등분산성 확인
_, pvalue = bartlett(*data)
print(f'{pvalue:.4f}')

F, pvalue = f_oneway(*data)
print(f'p-value가 {pvalue:.04f}이므로 {"기각" if pvalue < 0.05 else "채택"}')

0.2715 0.3380 0.1809
0.3515
p-value가 0.0000이므로 기각


### 4-2) 예제

세 가지 다른 교육 방법(A, B, C)을 사용하여 수험생들의 시험 성적을 개선시키는 효과를 평가하고자 한다. 30명의 학생들을 무작위로 세 그룹으로 배정하여 교육을 실시하였고, 시험을 보고 성적을 측정하였다.
일원배치법을 수행하여 그룹 간의 평균 성적 차이가 있는지 검정하시오.
- f값 (소수 둘째자리)
- p값 (소수 여섯째자리)
- 검정결과 출력

In [10]:
# 귀무가설(H0): 세 그룹(A, B, C) 간의 평균 성적 차이가 없다.
# 대립가설(H1 또는 Ha): 세 그룹(A, B, C) 간의 평균 성적 차이가 있다.

groupA = [85, 92, 78, 88, 83, 90, 76, 84, 92, 87]
groupB = [79, 69, 84, 78, 79, 83, 79, 81, 86, 88]
groupC = [75, 68, 74, 65, 77, 72, 70, 73, 78, 75]

fvalue, pvalue = f_oneway(groupA, groupB, groupC)

print(round(fvalue, 2))

print(format(pvalue,'.6f'))
print(f'p-value가 {pvalue:.06f}이므로 {"기각" if pvalue < 0.05 else "채택"}')

16.88
0.000018
p-value가 0.000018이므로 기각


# 2. 비모수 검정

- 모집단의 분포에 대해 제약(정규분포, 집단의 등분산 등)을 가하지 않고 실시하는 검정 방법
- 평균, 분산과 같은 모수 자체보다 분포 형태에 관한 검정을 실시함
- 관측 값들의 순위나 두 관측 값 사이의 부호, 중앙값 등을 이용해 검정

## 1) 카이제곱 적합도 검정
- 한 개 범주형 변수와 각 그룹별 비율과 특정 상수비가 같은지 검정
- **귀무가설(H0):** 변수의 분포가 기대 분포와 같다.
- **대립가설(H1):** 변수의 분포가 기대 분포와 같지 않다.

**특정 비율** A학급 40명의 혈액형 비율을 A, B, O, AB 각각 20%, 20%, 30%, 30%로 예상하였다. 실제 측정 결과 16, 16, 4, 4 명인 경우의 적합도 검정을 수행하여 보자.

In [21]:
# 귀무가설: 변수의 분포가 기대 분포와 같다
# 대립가설: 변수의 분포가 기대 분포와 같지 않다

data = {'blood_type': ['A']*16 + ['B']*16 + ['O']*4 + ['AB']*4}
data = pd.DataFrame(data)
data['blood_type'] = pd.Categorical(data['blood_type'], categories=['A', 'B', 'O', 'AB'])
# print(data.head())
observed = data['blood_type'].value_counts().sort_index().to_list()
expected = sum(observed) * np.array([0.2, 0.2, 0.3, 0.3]) # [8, 8, 12, 12]
print(observed, expected)

statistic, pvalue = chisquare(observed, expected)
print(f'{statistic:.04f} {pvalue:.04f}')
print('기각' if pvalue < 0.05 else '채택')

[16, 16, 4, 4] [ 8.  8. 12. 12.]
26.6667 0.0000
기각


4개의 범주에 대해 동일한 비율이라고 가정하고, 실제측정 한 표본 분포가 다음과 같을 때, 카이제곱 적합도 검정을 수행하여 보자.

In [22]:
# 귀무가설: 변수의 분포에 비율 차이가 없다.
# 대립가설: 변수의 분포에 비율 차이가 있다.

observed = [54, 46, 60, 40]
expected = sum(observed) * np.array([0.25]*4)
print(observed, expected)

statistic, pvalue = chisquare(observed, expected)
print(f'{statistic:.04f} {pvalue:.04f}')
print('기각' if pvalue < 0.05 else '채택')

[54, 46, 60, 40] [50. 50. 50. 50.]
4.6400 0.2001
채택


## 2) 카이제곱 동질성/독립성 검정

- **동질성 검정:** 각 집단이 범주형 변수에 대해 서로 유사한 성향을 갖는지 검정
  - 예) 남자 그룹, 여자 그룹의 표본을 각각 100명씩 추출하여, 각 표본의 핸드폰 모델 선호도를 조사함
  - **귀무가설(H0):** 집단 간 변수의 분포가 같다.
  - **대립가설(H1):** 집단 간 변수의 분포가 같지 않다.
- **독립성 검정:** 두 개 범주형 변수가 서로 독립인지 검정
  - 예) 일반인 200명을 임의 추출하고, 성별과 핸드폰 모델 선호도를 조사함
  - **귀무가설(H0):** 두 변수는 연관성이 없다(독립이다).
  - **대립가설(H1):** 두 변수는 연관성이 있다(독립이 아니다).

**다음의 교차표를 사용하여 성별별 핸드폰 모델 선호도 분포가 같은지 검정하라.**

           model_A  model_B  model_C
        M       10       40       50
        F       30       60       10

- 핸드폰 모델만 변수로 보고 성별은 비교 대상이 되는 그룹으로 해석
- 남자 그룹, 여자그룹의 표본을 각각 추출하여 각 표본의 핸드폰 모델 선호도를 조사함
- H0 : 성별별 핸드폰 모델 선호도 분포는 같다
- H1 : 성별별 핸드폰 모델 선호도 분포가 같지 않다       

In [23]:
crs_table = pd.DataFrame([[10, 40, 50], [30, 60, 10]], 
                         index=['M', 'F'], 
                         columns=['model_A', 'model_B', 'model_C'])
# print(crs_table)

statistic, pvalue, df, expected = chi2_contingency(crs_table)
print(f'{statistic:.04f} {pvalue:.04f} {df:.04f}\n')
print(pd.DataFrame(expected), '\n')
print('기각' if pvalue < 0.05 else '채택')

40.6667 0.0000 2.0000

      0     1     2
0  20.0  50.0  30.0
1  20.0  50.0  30.0 

기각


**카이제곱 독립성 검정 data_chi.csv 파일을 사용하여 당뇨와 비만 사이의 관계가 독립인지 검정하라.**

In [24]:
df = pd.read_csv('./data_02/data_chi.csv')
# print(df.head())

# 카이제곱 분석을 위한 분할표 만들기
crs_table = pd.crosstab(df['당뇨 여부'], df['비만 여부'])
# print(crs_table)

# chisquare 검정 
statistic, pvalue, df, expected = chi2_contingency(crs_table)
print(f'{statistic:.04f} {pvalue:.04f} {df:.04f}\n')
print(pd.DataFrame(expected), '\n')
print('기각' if pvalue < 0.05 else '채택')

0.0000 1.0000 1.0000

       0      1
0  62.16  21.84
1  11.84   4.16 

채택


**아내와 남편의 집안일 관계가 독립인지 검정하라.**

In [25]:
# 귀무가설: 집안일의 종류와 수행하는 사람의 관계는 독립이다.
# 대립가설: 집안일의 종류와 수행하는 사람의 관계가 독립이 아니다.

df = pd.read_csv('data_02/housetasks.csv', index_col=0)
# print(df.head())

# 카이제곱 독립성 검정
statistic, pvalue, df, expected = chi2_contingency(df)
print(f'{statistic:.04f} {pvalue:.04f} {df:.04f}\n')
print(pd.DataFrame(expected), '\n')
print('기각' if pvalue < 0.05 else '채택')

1944.4562 0.0000 36.0000

            0          1          2          3
0   60.550459  25.633028  38.449541  51.366972
1   52.637615  22.283257  33.424885  44.654243
2   37.155963  15.729358  23.594037  31.520642
3   48.165138  20.389908  30.584862  40.860092
4   41.972477  17.768349  26.652523  35.606651
5   38.876147  16.457569  24.686353  32.979931
6   41.284404  17.477064  26.215596  35.022936
7   33.027523  13.981651  20.972477  28.018349
8   47.821101  20.244266  30.366399  40.568234
9   38.876147  16.457569  24.686353  32.979931
10  47.821101  20.244266  30.366399  40.568234
11  56.766055  24.030963  36.046445  48.156537
12  55.045872  23.302752  34.954128  46.697248 

기각


## 3) 피셔의 정확 검정

In [26]:
# 귀무가설: 약과 효과의 관계는 독립이다. 
# 대립가설: 약과 효과의 관계가 독립이 아니다.

In [27]:
# [1] 분할표 생성
df = pd.DataFrame([[1, 6], [5, 2]])
df.columns = ['가짜 약','진짜 약']
df.index = ['효과있음', '효과없음']
print(df.head())

# [2] 피셔의 정확 검정
oddsratio, pvalue = fisher_exact(df)
print(f'{oddsratio:.04f} {pvalue:.04f}')
print('기각' if pvalue < 0.05 else '채택')

      가짜 약  진짜 약
효과있음     1     6
효과없음     5     2
0.0667 0.1026
채택


## 4) Wilcoxon Signed Rank Test

- 1표본/대응표본 T 검정에서 정규성 가정이 만족되지 않을 때 사용하는 비모수 검정법으로 중앙값에 관한 결과를 얻을 수 있음
- 가정: 표본은 동일한 모집단에서 추출되어야 하며, 임의, 독립적으로 추출되어야 한다.

### 4-1) 일표본(One sample)

**중량이 100g 으로 표기된 닭가슴살 제품이 100g이라고 할 수 있는가?**
- 동일한 회사 제품을 임의로 9개 표본 추출하였음
- 통계적 유의수준은 0.05로 사용함

In [28]:
# 귀무가설: 닭 가슴살 중량의 중앙값은 100g이다.
# 대립가설: 닭 가슴살 중량의 중앙값은 100g이 아니다.

df = pd.read_csv('data_02/chicken_breast.csv')
# print(df.head())

at_s, at_p = wilcoxon(df.weight - 100, alternative='two-sided')
print(f'statistic: {at_s:.04f} p-value: {at_p:.04f}')
print('기각' if at_p < 0.05 else '채택\n')

al_s, al_p = wilcoxon(df.weight - 100, alternative='less')
print(f'statistic: {al_s:.04f} p-value: {al_p:.04f}')
print('기각' if al_p < 0.05 else '채택\n')

ag_s, ag_p = wilcoxon(df.weight - 100, alternative='greater')
print(f'statistic: {ag_s:.04f} p-value: {ag_p:.04f}')
print('기각' if ag_p < 0.05 else '채택\n')

statistic: 10.0000 p-value: 0.1641
채택

statistic: 35.0000 p-value: 0.9355
채택

statistic: 35.0000 p-value: 0.0820
채택



### 4-2) 대응표본(Paired)

In [29]:
# 귀무가설: 다이어트 후 - 다이어트 전 몸무게 중앙값은 0과 같다 
# 대립가설: 다이어트 후 - 다이어트 전 몸무게 중앙값은 0과 다르다

df = pd.read_csv('data_02/diet_result.csv')
# print(df.head())

at_s, at_p = wilcoxon(df.after, df.before, alternative='two-sided')
print(f'statistic: {at_s:.04f} p-value: {at_p:.04f}')
print('기각' if at_p < 0.05 else '채택\n')

al_s, al_p = wilcoxon(df.after, df.before, alternative='less')
print(f'statistic: {al_s:.04f} p-value: {al_p:.04f}')
print('기각' if al_p < 0.05 else '채택\n')

ag_s, ag_p = wilcoxon(df.after, df.before, alternative='greater')
print(f'statistic: {ag_s:.04f} p-value: {ag_p:.04f}')
print('기각' if ag_p < 0.05 else '채택\n')

statistic: 1.0000 p-value: 0.0156
기각
statistic: 1.0000 p-value: 0.0078
기각
statistic: 1.0000 p-value: 0.9961
채택



## 5) Wilcoxon Rank Sum Test

- 독립2표본 T 검정에서 정규성 가정이 만족되지 않을 때 사용하는 비모수 검정법으로 두 표본의 중위수를 비교하는 데에 쓰임

### 5-1) 2표본 독립

**A사 닭가슴살 제품의 중량과 B사 닭가슴살 제품의 중량이 차이가 있는지 확인하라.**
- A사 닭가슴살은 40개의 표본, B사 닭가슴살은 20개의 표본이 있으며
- 각각 독립적이고 임의로 추출했다
- 통계적 유의수준은 0.05 사용

In [30]:
# 귀무가설: A사 닭가슴살 중량과 B사 닭가슴살 중량의 차이가 없다.
# 대립가설: A사 닭가슴살 중량과 B사 닭가슴살 중량의 차이가 있다.

df = pd.read_csv('data_02/chicken_weight_AB.csv')
# print(df.head())

gA = df.loc[df['company']=='A', 'weight']
gB = df.loc[df['company']=='B', 'weight']

_, pA = shapiro(gA)  # 정규성 만족
_, pB = shapiro(gB)  # 정규성 만족하지 않음
print(f'{pA:.04f}, {pB:.04f}')

at_s, at_p = ranksums(gA, gB, alternative='two-sided')
print(f'statistic: {at_s:.04f} p-value: {at_p:.04f}')
print('기각' if at_p < 0.05 else '채택\n')

al_s, al_p = ranksums(gA, gB, alternative='less')
print(f'statistic: {al_s:.04f} p-value: {al_p:.04f}')
print('기각' if al_p < 0.05 else '채택\n')

ag_s, ag_p = ranksums(gA, gB, alternative='greater')
print(f'statistic: {ag_s:.04f} p-value: {ag_p:.04f}')
print('기각' if ag_p < 0.05 else '채택\n')

0.5676, 0.0006
statistic: -4.8612 p-value: 0.0000
기각
statistic: -4.8612 p-value: 0.0000
기각
statistic: -4.8612 p-value: 1.0000
채택



### 5-2) Mann-Whitney U test

In [31]:
at_s, at_p = mannwhitneyu(gA, gB, alternative='two-sided')
print(f'statistic: {at_s:.04f} p-value: {at_p:.04f}')
print('기각' if at_p < 0.05 else '채택\n')

al_s, al_p = mannwhitneyu(gA, gB, alternative='less')
print(f'statistic: {al_s:.04f} p-value: {al_p:.04f}')
print('기각' if al_p < 0.05 else '채택\n')

ag_s, ag_p = mannwhitneyu(gA, gB, alternative='greater')
print(f'statistic: {ag_s:.04f} p-value: {ag_p:.04f}')
print('기각' if ag_p < 0.05 else '채택\n')

statistic: 90.0000 p-value: 0.0000
기각
statistic: 90.0000 p-value: 0.0000
기각
statistic: 90.0000 p-value: 1.0000
채택



## 6) Kruskal-Wallis H-test

- 분산분석에서 정규성 가정이 만족되지 않을 때 사용하는 비모수 검정법으로 중앙값에 관한 결과를 얻을 수 있음
- 가정: 표본은 독립적이며, 측정값은 최소 순서형 변수이다(대소 비교가 가능해야 한다).

**A, B, C 고등학교 학생들의 하루 공부 시간을 조사했을 때,고등학교 간에 공부 시간이 차이가 있는지 확인하라.**
- 통계적 유의수준은 0.05

In [32]:
# 귀무가설: A,B,C 고등학교 학생들의 하루 공부시간에 차이가 없다.
# 대립가설: A,B,C 고등학교 학생들의 하루 공부시간에 차이가 있다.

df = pd.read_csv('data_02/high_school.csv')
# print(df.head())

gA = df.loc[df.group =='A', 'time']
gB = df.loc[df.group =='B', 'time']
gC = df.loc[df.group =='C', 'time']
print(len(gA), len(gB), len(gC))

_, pA = shapiro(gA)  # 정규성 만족
_, pB = shapiro(gB)  # 정규성 만족
_, pC = shapiro(gC)  # 정규성 만족
print(f'A={pA:.4f} B={pB:.4f} C={pC:.4f}')

statistic, pvalue = kruskal(gA, gB, gC)
print(f'statistic:{statistic:.4f} pvalue: {pvalue:.4f}')

statistic, pvalue = f_oneway(gA, gB, gC)
print(f'statistic:{statistic:.4f} pvalue: {pvalue:.4f}')

20 30 15
A=0.4412 B=0.6307 C=0.2647
statistic:34.1434 pvalue: 0.0000
statistic:1394.1870 pvalue: 0.0000
